In [1]:
import json
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, accuracy_score)

In [2]:
file_path = 'data/Bắc Bộ.csv'
file_name = file_path.split('/')[-1][:-4]
log_file = 'logs/{}.txt'.format(file_name)
df = pd.read_csv(file_path)

In [3]:
df.head()

,Unnamed: 0,Time,Humidity,Dew point,Day,Month,Year,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather
0,5755,00:00,88.0,18.0,1,1,2021,12.0,0.0,32.0,1026.0,7.0,12.0,38.0,Partly cloudy
1,5756,03:00,88.0,18.0,1,1,2021,11.0,0.0,37.0,1026.0,10.0,15.0,23.5,Partly cloudy
2,5757,06:00,83.0,17.0,1,1,2021,10.0,0.0,33.0,1026.0,10.0,17.0,15.3,Partly cloudy
3,5758,09:00,73.0,16.0,1,1,2021,13.0,0.0,22.0,1027.0,12.0,15.0,12.1,Partly cloudy
4,5759,12:00,65.0,16.0,1,1,2021,17.0,0.0,12.0,1024.0,10.0,12.0,22.4,Partly cloudy


In [4]:
label = df['Weather']
features = df

In [5]:
#remove data
features = features[:-1]
label = label[1:]

In [6]:
data = features
data['Label'] = label.values

/tmp/ipykernel_3943366/1655283038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Label'] = label.values


In [7]:
RAINY_LABEL = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain',
             'Patchy light rain with thunder','Patchy rain possible','Heavy rain','Heavy rain at times',
             'Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast','Torrential rain shower']
RAINY_MONTH = [5,6,7,8,9,10,11]
MORNING_HOUR = [6,9,12,15]

In [8]:
for i, value in enumerate(data.Label):
    if value in RAINY_LABEL:
        data.Label[i] = 'yes'
    else:
        data.Label[i] = 'no'

/tmp/ipykernel_3943366/4224172725.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Label[i] = 'no'
/data4/thang/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_3943366/4224172725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Label[i] = 'yes'


KeyboardInterrupt: 

In [ ]:
# for i, value in enumerate(data.Time):
#     hour = value.split(':')[0]
#     data.Time[i] = hour

In [ ]:
for i, value in enumerate(data.Weather):
    if value in RAINY_LABEL:
        data.Weather[i] = 'rainy'
    else:
        data.Weather[i] = 'sunny'

/tmp/ipykernel_2296341/812369229.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Weather[i] = 'sunny'
/tmp/ipykernel_2296341/812369229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Weather[i] = 'rainy'


In [ ]:
# for i, value in enumerate(data.Month):
#     if int(value) in RAINY_MONTH:
#         data.Month[i] = 'rainy_month'
#     else:
#         data.Month[i] = 'sunny_month'

In [ ]:
# remove columns
# drop_columns = ['Day', 'Year', 'Humidity', 'Pressure']
drop_columns = []
data = data.drop(drop_columns, axis=1)
data.head()

,Unnamed: 0,Time,Humidity,Dew point,Day,Month,Year,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather,Label
0,0,00:00,65.0,19.0,1,1,2021,23.0,0.0,26.0,1012.0,18.0,29.0,12.8,sunny,no
1,1,03:00,69.0,19.0,1,1,2021,22.0,0.0,32.0,1011.0,17.0,26.0,11.4,sunny,no
2,2,06:00,78.0,18.0,1,1,2021,21.0,0.0,43.0,1012.0,15.0,25.0,8.0,sunny,no
3,3,09:00,55.0,19.0,1,1,2021,24.0,0.0,36.0,1013.0,19.0,24.0,15.0,sunny,no
4,4,12:00,30.0,14.0,1,1,2021,27.0,0.0,28.0,1012.0,19.0,22.0,14.1,sunny,no


In [ ]:
def convert_category2numeric(data, columns_name):
    for col in columns_name:
        data = pd.concat((data, pd.get_dummies(data[col], prefix=col)), axis=1)
        data = data.drop([col], axis=1)
    return data

In [ ]:
data = convert_category2numeric(data, ['Time', 'Weather', 'Month'])
data = pd.concat((data, pd.get_dummies(data['Label'], prefix='Label', drop_first=True)), axis=1)
data = data.drop(['Label'], axis=1)
data.head()

,Unnamed: 0,Humidity,Dew point,Day,Year,Temperature,Rain,Cloud,Pressure,Wind,...,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Label_yes
0,0,65.0,19.0,1,2021,23.0,0.0,26.0,1012.0,18.0,...,0,0,0,0,0,0,0,0,0,0
1,1,69.0,19.0,1,2021,22.0,0.0,32.0,1011.0,17.0,...,0,0,0,0,0,0,0,0,0,0
2,2,78.0,18.0,1,2021,21.0,0.0,43.0,1012.0,15.0,...,0,0,0,0,0,0,0,0,0,0
3,3,55.0,19.0,1,2021,24.0,0.0,36.0,1013.0,19.0,...,0,0,0,0,0,0,0,0,0,0
4,4,30.0,14.0,1,2021,27.0,0.0,28.0,1012.0,19.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
all_columns = data.columns.to_list()
Y_colums = ['Label_yes']
X_colums = [x for x in all_columns if x not in Y_colums]

In [ ]:
train_set, test_set = train_test_split(data, test_size=0.2)

In [ ]:
Xtrain = train_set[X_colums]
ytrain = train_set[Y_colums]

# building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit()
log_reg.summary()

/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully.
         Current function value: inf
         Iterations 8


/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Label_yes   No. Observations:               145042
Model:                          Logit   Df Residuals:                   145011
Method:                           MLE   Df Model:                           30
Date:                Mon, 25 Jul 2022   Pseudo R-squ.:                     inf
Time:                        15:16:41   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Unnamed: 0    -1.357e-07   1.44e-07     -0.940      0.347   -4.19e-07    1.47e-07
Humidity          0.0034      0.001      4.615      0.000       0.002       0.005
Dew point         0.0111      0.003      3.748      0.000       0.005       0.017
Day              -0.0048      0.001     -5.505      0.000      -0.006      -0.003
Year             -0.0321        nan        nan        nan         nan         nan
Temperature       0.0781      0.003     26.079      0.000       0.072       0.084
Rain              1.5190      0.025     60.718      0.000       1.470       1.568
Cloud             0.0557      0.000    135.061      0.000       0.055       0.057
Pressure          0.0576      0.003     20.633      0.000       0.052       0.063
Wind             -0.0197      0.006     -3.129      0.002      -0.032      -0.007
Gust              0.0235      0.004      5.250      0.000       0.015       0.032
Dir               0.0002   8.62e-05      1.958      0.050   -2.06e-07       0.000
Time_00:00       -0.4710    2.7e+05  -1.74e-06      1.000   -5.29e+05    5.29e+05
Time_03:00        0.0157    2.7e+05   5.83e-08      1.000   -5.29e+05    5.29e+05
Time_06:00       -0.2814    2.7e+05  -1.04e-06      1.000   -5.29e+05    5.29e+05
Time_09:00        0.4436    2.7e+05   1.64e-06      1.000    -5.3e+05     5.3e+05
Time_12:00       -0.1581    2.7e+05  -5.86e-07      1.000   -5.29e+05    5.29e+05
Time_15:00        0.7030    2.7e+05   2.61e-06      1.000   -5.29e+05    5.29e+05
Time_18:00       -0.4125    2.7e+05  -1.53e-06      1.000   -5.29e+05    5.29e+05
Time_21:00        0.1607    2.7e+05   5.96e-07      1.000   -5.29e+05    5.29e+05
Weather_rainy     0.1310   1.29e+05   1.01e-06      1.000   -2.53e+05    2.53e+05
Weather_sunny    -0.1311   1.29e+05  -1.01e-06      1.000   -2.53e+05    2.53e+05
Month_1          -0.2057   3.74e+05   -5.5e-07      1.000   -7.33e+05    7.33e+05
Month_2          -0.2981   3.74e+05  -7.97e-07      1.000   -7.33e+05    7.33e+05
Month_3          -0.0694   3.74e+05  -1.86e-07      1.000   -7.33e+05    7.33e+05
Month_4           0.2011   3.74e+05   5.38e-07      1.000   -7.33e+05    7.33e+05
Month_5           0.0563   3.74e+05   1.51e-07      1.000   -7.33e+05    7.33e+05
Month_6           0.1106   3.74e+05   2.96e-07      1.000   -7.33e+05    7.33e+05
Month_7           0.2419   3.74e+05   6.47e-07      1.000   -7.33e+05    7.33e+05
Month_8           0.1936   3.74e+05   5.18e-07      1.000   -7.33e+05    7.33e+05
Month_9           0.3405   3.74e+05   9.11e-07      1.000   -7.33e+05    7.33e+05
Month_10          0.0066   3.74e+05   1.76e-08      1.000   -7.33e+05    7.33e+05
Month_11         -0.1647   3.74e+05  -4.41e-07      1.000   -7.33e+05    7.33e+05
Month_12         -0.4128   3.74e+05   -1.1e-06      1.000   -7.33e+05    7.33e+05
=================================================================================
"""

In [ ]:
Xtest = test_set[all_columns[:-1]]
ytest = test_set[all_columns[-1]]

yhat = log_reg.predict(Xtest)
prediction = list(map(round, yhat))
cm = confusion_matrix(ytest, prediction) 
print ("Confusion Matrix : \n", cm)

# accuracy score of the model
print('Test accuracy = ', accuracy_score(ytest, prediction))

Confusion Matrix : 
 [[17474  3190]
 [ 2885 12712]]
Test accuracy =  0.8324646314221892


In [ ]:
with open(log_file, 'w') as fp:
    fp.write(str(log_reg.summary()))
    fp.write("\n")
    fp.write('Test accuracy = {}'.format(accuracy_score(ytest, prediction)))
log_reg.save('models/{}.pickle'.format(file_name))